# Combining Data

The purpose of this notebook is to make one giant CSV module, out of the many image files in the dataset, which are scattered across different folders.

## Imports

In [2]:
import pandas as pd
import glob

## Creating a DataFrame for the Images in Each Folder

This is a **binary classification** problem. Here is what the labels I'm assigning mean:

- 0 - there is NO FIRE detected in the image
- 1 - there is a FIRE detected in the image

In [9]:
def make_df(folder):
    """Returns a pandas.DataFrame object from the folder."""
    with open(folder) as f:
        ls_data = f.readlines()
        df = pd.DataFrame(ls_data, columns['Folcer', 'filename', 'label'])
        return df

In [10]:
ls_df = list()

# add all the images with no fire
for filepath in glob.glob('Fire-Detection-Image-Dataset/Normal Images*'):
    df_no_fire = make_df(filepath)
    ls_df.append(df_no_fire)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc0 in position 23: invalid start byte